## Examples

In [1]:
!python -m pip install numpy

In [2]:
!python -m pip uninstall -y changepoint_online

Found existing installation: changepoint-online 0.0.1
Uninstalling changepoint-online-0.0.1:
  Successfully uninstalled changepoint-online-0.0.1


In [3]:
!python -m pip install 'git+https://github.com/grosed/changepoint.online/#egg=changepoint.online&subdirectory=python/package'

  Cloning https://github.com/grosed/changepoint.online/ to /tmp/pip-install-cbfgfmgz/changepoint-online_843eb680b3a7466d955057d9de29f8f0
  Running command git clone --filter=blob:none --quiet https://github.com/grosed/changepoint.online/ /tmp/pip-install-cbfgfmgz/changepoint-online_843eb680b3a7466d955057d9de29f8f0
  Resolved https://github.com/grosed/changepoint.online/ to commit 1c25964d0b24c27967bada6574dbd45b469adf8c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for changepoint.online: filename=changepoint_online-0.0.1-py3-none-any.whl size=15772 sha256=4e885a91aa26ce111b6dbc8a4dc25dcae8afdbbe37fc663128a6261dcecbbf77
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5exoxqn/wheels/35/ed/e6/7ac6c98a8e36d6ca8ae017fa590878bafa3b330c405334d740
Successfully built changepoint.online


In [4]:
from changepoint_online import *

In [5]:
import numpy as np

In [6]:
np.random.seed(0)
Y = np.concatenate((np.random.normal(loc=0.0, scale=1.0, size=5000), np.random.normal(loc=10.0, scale=1.0, size=5000)))

### Guassian

In [7]:
detector = Focus(Guassian)

In [8]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.threshold() >= threshold:
        break

In [9]:
detector.changepoint()

{'stopping_time': 5001, 'changepoint': 5000}

### Poisson

In [10]:
np.random.seed(0)
Y = np.concatenate((np.random.poisson(lam=4.0, size=5000), np.random.poisson(lam=4.2, size=5000)))

In [11]:
detector = Focus(Poisson)

In [12]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.threshold() >= threshold:
        break

In [13]:
detector.changepoint()

{'stopping_time': 6488, 'changepoint': 4973}

### Gamma

In [14]:
np.random.seed(0)
Y = np.concatenate((np.random.gamma(4.0, scale=3.0, size=5000), np.random.gamma(4.0, scale=6.0, size=5000)))

In [15]:
detector = Focus(Gamma(1))

In [16]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.threshold() >= threshold:
        break

In [17]:
detector.changepoint()

{'stopping_time': 5033, 'changepoint': 5002}